# Test the ENC-DEC model

In [35]:
import numpy as np
import dataloader
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import models
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision
import os
import time
from torch_geometric.nn import GATv2Conv, global_mean_pool
# reload library
import importlib
import cv2
#import utils as ut
import pandas as pd
import DataDLC
from torch_geometric.data import Data, DataLoader
import tqdm


# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

In [36]:
importlib.reload(dataloader)
importlib.reload(models)
importlib.reload(DataDLC)

<module 'DataDLC' from 'c:\\Users\\Usuario\\Documents\\Documents\\MVA\\Stage\\DLCProject\\Code\\GitHubRep\\Behavioral_Tagging_of_Mice_in_multiple_Mice_dataset_using_Deep_Learning\\src\\DataDLC.py'>

### Load data

---

**This is only to create an eassy graph to test the build_graph_2 function**



In [37]:
if False:
    data_dlc.coords = data_dlc.coords.iloc[range(3), :]
    data_dlc.n_frames = 3
    data_dlc.save(r'C:\Users\Usuario\Documents\Documents\MVA\Stage\DLCProject\Data\DataLoaderTestFormat\output1.h5')


## Create dataset

**Obs:** This is just a test to see if the model is working.

In [38]:
# print numpy version
print(np.__version__)

# print pytorch version
print(torch.__version__)



1.26.4
2.4.0+cpu


In [39]:
importlib.reload(dataloader)
importlib.reload(DataDLC)

<module 'DataDLC' from 'c:\\Users\\Usuario\\Documents\\Documents\\MVA\\Stage\\DLCProject\\Code\\GitHubRep\\Behavioral_Tagging_of_Mice_in_multiple_Mice_dataset_using_Deep_Learning\\src\\DataDLC.py'>

In [40]:

dataloader.reload_module()

In [41]:
# deactivate warnings
if False:
    import warnings
    warnings.filterwarnings("ignore")


    data_loader = dataloader.DLCDataLoader(r'C:\Users\Usuario\Documents\Documents\MVA\Stage\DLCProject\Data\DataLoaderTrainTest', batch_size=1, num_workers=0, device='cpu', window_size=3, stride = 1, build_graph=True)
    #data_loader = dataloader.DLCDataLoader(r'C:\Users\Usuario\Documents\Documents\MVA\Stage\DLCProject\Data\DataLoaderTestDataset', batch_size=1, num_workers=0, device='cpu', window_size=3, stride = 1, build_graph=True, behaviour='contacts generaux (R + V) active')
    

---

## Load Dataset

*Graphs were already created and saved in the data folder*

In [42]:
entire_dataset = torch.load(r'c:\Users\Usuario\Documents\Documents\MVA\Stage\DLCProject\Data\DataLoader_Baseline_models\dataset_w5-001.pt')

In [43]:
print('The dataset has %d samples' % len(entire_dataset))

The dataset has 155042 samples


In [44]:
entire_dataset[0].behaviour[1]

0.0

In [45]:
entire_dataset[-1].frame_mask

tensor([0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3,
        4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2,
        3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1,
        2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0,
        1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4,
        0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3,
        4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2,
        3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4], dtype=torch.int32)

In [46]:
dataset = entire_dataset.copy()

In [47]:
# Only keep the 'General_Contact' behaviour.

for i in range(len(dataset)):
    dataset[i].y = torch.tensor([dataset[i].behaviour[1]], dtype=torch.long)
    dataset[i].frame_mask = (dataset[i].frame_mask == 2).float()

In [48]:
entire_dataset[0].frame_mask

tensor([0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.])

In [58]:
len(dataset[0].frame_mask)

180

#### Split Train-Test

**Shuffle**

In [95]:
# Suffle the dataset
np.random.seed(0)
np.random.shuffle(dataset)

In [96]:
# Train / Test split
train_size = int(0.8 * len(dataset))

train_dataset = dataset[:train_size]
test_dataset = dataset[train_size:]

In [97]:
print('The train dataset has %d samples' % len(train_dataset))
print('The test dataset has %d samples' % len(test_dataset))


The train dataset has 124033 samples
The test dataset has 31009 samples


In [125]:
batch_size = 1

# Create the dataloaders for train, validation and test
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [100]:
for i, data in enumerate(train_loader):
    first_batch = data
    break

In [101]:
first_batch

DataBatch(x=[2880, 4], edge_index=[2, 54240], file=[16], frame_mask=[2880], behaviour=[16], y=[16], batch=[2880], ptr=[17])

In [102]:
first_batch.x[first_batch.batch == 0][first_batch.frame_mask[first_batch.batch == 0] == 1].shape

torch.Size([36, 4])

---

### Graph Classifier

In [103]:
importlib.reload(models)

<module 'models' from 'c:\\Users\\Usuario\\Documents\\Documents\\MVA\\Stage\\DLCProject\\Code\\GitHubRep\\Behavioral_Tagging_of_Mice_in_multiple_Mice_dataset_using_Deep_Learning\\src\\models.py'>

In [104]:
graphencoder = models.GATEncoder(nout = 64, nhid=32, attention_hidden=2, n_in=4, dropout=0.5)
class_head = models.ClassificationHead(n_latent = 64, nhid=64, nout=2)

In [105]:
# One try to see if the model is working
embbed = graphencoder(first_batch.x, first_batch.edge_index, first_batch.frame_mask)


In [106]:
embbed.shape

torch.Size([2880, 64])

In [107]:
embbed = global_mean_pool(embbed, first_batch.batch)

In [108]:
embbed.shape

torch.Size([16, 64])

In [109]:
out = class_head(embbed)

In [110]:
out.shape

torch.Size([16, 2])

In [133]:
model = models.GraphClassifier(graphencoder, class_head)

In [112]:
out = model.forward(first_batch)

In [113]:
print(out.shape)

torch.Size([16, 2])


In [114]:
out

tensor([[0.5493, 0.4507],
        [0.5492, 0.4508],
        [0.5496, 0.4504],
        [0.5508, 0.4492],
        [0.5522, 0.4478],
        [0.5479, 0.4521],
        [0.5489, 0.4511],
        [0.5483, 0.4517],
        [0.5443, 0.4557],
        [0.5451, 0.4549],
        [0.5489, 0.4511],
        [0.5486, 0.4514],
        [0.5501, 0.4499],
        [0.5492, 0.4508],
        [0.5470, 0.4530],
        [0.5499, 0.4501]], grad_fn=<SoftmaxBackward0>)

In [115]:
print(len(train_loader))

7753


## Benchmark model 

In [116]:
batch = next(iter(train_loader))

x, edge_index, batch, frame_mask = batch.x, batch.edge_index, batch.batch, batch.frame_mask

In [117]:
frame_mask

tensor([0., 0., 1.,  ..., 1., 0., 0.])

torch.Size([2880, 4])


IndexError: tensors used as indices must be long, int, byte or bool tensors

In [145]:

model = models.SimpleMLPforGraph(n_in=144, n_hid=128, n_out=2)

# Trainning

In [146]:
# Trainning loop
num_epochs = 10
lr = 0.001
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
writer = SummaryWriter(log_dir='runs/gcn_action_detection')  # TensorBoard writer


In [147]:

# Training Loop
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for i, data in tqdm.tqdm(enumerate(train_loader)):
        optimizer.zero_grad()
        outputs = model(data).unsqueeze(0)
        labels = data.y
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
        train_loss += loss.item()
        predicted = outputs.argmax(dim=1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
        # Log training loss and accuracy at each step
        if i % 10 == 0:  # Log every 10 iterations, adjust as needed
            writer.add_scalar('Loss/Train', loss.item(), epoch * len(train_loader) + i)
            writer.add_scalar('Accuracy/Train', correct / total, epoch * len(train_loader) + i)
            #print(f"Epoch {epoch+1}, Step {i}, Loss: {loss.item()}, Accuracy: {correct / total}")

    train_accuracy = correct / total
    avg_train_loss = train_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss}, Training Accuracy: {train_accuracy}")

    # Validation phase
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for val_data in tqdm.tqdm(test_loader):
            val_outputs = model(val_data).unsqueeze(0)
            val_labels = val_data.y
            val_loss += criterion(val_outputs, val_labels).item()
            val_predicted = val_outputs.argmax(dim=1)
            correct += (val_predicted == val_labels).sum().item()
            total += val_labels.size(0)
    
    val_accuracy = correct / total
    avg_val_loss = val_loss / len(test_loader)
    
    print(f"Validation Loss: {avg_val_loss}, Validation Accuracy: {val_accuracy}")

    # Log validation metrics
    writer.add_scalar('Loss/Validation', avg_val_loss, epoch)
    writer.add_scalar('Accuracy/Validation', val_accuracy, epoch)

# Close the TensorBoard writer
writer.close()


124033it [56:13, 36.77it/s] 


Epoch 1, Training Loss: 0.562334276328933, Training Accuracy: 0.7284029250280167


100%|██████████| 31009/31009 [00:57<00:00, 537.79it/s]


Validation Loss: 0.5308780824410742, Validation Accuracy: 0.7653906930246057


124033it [1:32:53, 22.25it/s] 


...

Epoch 3, Training Loss: 0.523211862823944, Training Accuracy: 0.7783170607822112


100%|██████████| 31009/31009 [01:53<00:00, 273.39it/s]


Validation Loss: 0.5034721102385924, Validation Accuracy: 0.8010577574252636


112948it [5:23:37, 122.29it/s]

: 

In [ ]:
    for i, data in tqdm.tqdm(enumerate(train_loader)):
        optimizer.zero_grad()
        outputs = model(data)
        labels = data.y
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
        train_loss += loss.item()
        predicted = outputs.argmax(dim=1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
    
        # Log training loss and accuracy at each step
        if i % 10 == 0:  # Log every 10 iterations, adjust as needed
            writer.add_scalar('Loss/Train', loss.item(), epoch * len(train_loader) + i)
            writer.add_scalar('Accuracy/Train', correct / total, epoch * len(train_loader) + i)
            #print(f"Epoch {epoch+1}, Step {i}, Loss: {loss.item()}, Accuracy: {correct / total}")

    train_accuracy = correct / total
    avg_train_loss = train_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss}, Training Accuracy: {train_accuracy}")

    # Validation phase
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for val_data in test_loader:
            val_outputs = model(val_data)
            val_labels = val_data.y
            val_loss += criterion(val_outputs, val_labels).item()
            val_predicted = val_outputs.argmax(dim=1)
            correct += (val_predicted == val_labels).sum().item()
            total += val_labels.size(0)
    
    val_accuracy = correct / total
    avg_val_loss = val_loss / len(test_loader)
    
    print(f"Validation Loss: {avg_val_loss}, Validation Accuracy: {val_accuracy}")

    # Log validation metrics
    writer.add_scalar('Loss/Validation', avg_val_loss, epoch)
    writer.add_scalar('Accuracy/Validation', val_accuracy, epoch)

# Close the TensorBoard writer
writer.close()


# Trainning loop
epochs = 50
lr = 0.001
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

for epoch in range(epochs):
    running_loss = 0.0
    las
    model.train()
    for i, data in enumerate(train_loader):
        optimizer.zero_grad()
        out = model.forward(data)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100
        

In [ ]:
out = GraphAE.forward(data_loader.dataset[0].x, data_loader.dataset[0].edge_index, data_loader.dataset[0].frame_mask)

In [ ]:
# Loss and optimizer
loss = GraphAE.loss(data_loader.dataset[0].x, out)

In [ ]:
print(loss)

# Trainning loop

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
encoder = models.GATEncoder(nout = 64, nhid=16, attention_hidden=2, n_in=3, dropout=0.5).to(device)
print(encoder)
decoder = models.GATDecoder(n_latent=64, n_hidden=16, n_out=3).to(device)
print(decoder)
model = models.GraphAE(encoder, decoder).to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-3)
data = data_loader.dataset

for epoch in range(100):
    model.train()
    optimizer.zero_grad()
    for i in range(len(data)):

        out = model(data[i].x.to(device), data[i].edge_index.to(device), data[i].frame_mask.to(device))
        loss = model.loss(data[i].x, out)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch}, Loss {loss.item()}')

model.eval()
out = model(data[0].x, data[0].edge_index, data[0].frame_mask)

In [ ]:
print(out[0][0].shape)

In [ ]:
numpyout = out[0][0].detach().numpy()

In [ ]:
# Plot the output of the model, the first dimension are the points, and the second one is the x and y coordinates
plt.scatter(numpyout[:,0], numpyout[:,1])


In [ ]:
print(data_loader.dataset[0].x.shape)
print(x[0].shape)
print(x[1].shape)

In [ ]:
print(data_loader.dataset[0].x.shape)
print(x[0].shape)
print(x[1].shape)

In [ ]:
contacts = behaviour.iloc[:,1]

---